In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem

def get_ecfc(smiles_list, radius=2, nBits=2048, useCounts=True):
    """
    Calculates the ECFP fingerprint for given SMILES list
    
    :param smiles_list: List of SMILES
    :type smiles_list: list
    :param radius: The ECPF fingerprints radius.
    :type radius: int
    :param nBits: The number of bits of the fingerprint vector.
    :type nBits: int
    :param useCounts: Use count vector or bit vector.
    :type useCounts: bool
    :returns: The calculated ECPF fingerprints for the given SMILES
    :rtype: Dataframe
    """     
    
    ecfp_fingerprints=[]
    erroneous_smiles=[]
    for smiles in smiles_list:
        mol=Chem.MolFromSmiles(smiles)
        if mol is None:
            ecfp_fingerprints.append([None]*nBits)
            erroneous_smiles.append(smiles)
        else:
            mol=Chem.AddHs(mol)
            if useCounts:
                ecfp_fingerprints.append(list(AllChem.GetHashedMorganFingerprint(mol, radius, nBits)))  
            else:    
                ecfp_fingerprints.append(list(AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits).ToBitString()))  
    
    # Create dataframe of fingerprints
    df_ecfp_fingerprints = pd.DataFrame(data = ecfp_fingerprints, index = smiles_list)
    # Remove erroneous data
    if len(erroneous_smiles)>0:
        print("The following erroneous SMILES have been found in the data:\n{}.\nThe erroneous SMILES will be removed from the data.".format('\n'.join(map(str, erroneous_smiles))))           
        df_ecfp_fingerprints = df_ecfp_fingerprints.dropna(how='any')    
    
    return df_ecfp_fingerprints

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd

In [3]:
#Get and Arrange data
import pandas as pd
df_data= pd.read_csv('all_data.csv')

train_data = df_data[df_data['data_type'] == 0]
test1_data = df_data[df_data['data_type'] == 1]
test2_data = df_data[df_data['data_type'] == 2]

In [4]:
test2_data.head()

,reaction_id,data_package_id,bond_type,functional_group_stoichiometry,reactant_smiles,product_smiles,reactant_inchiKey,product_inchiKey,reactant_solubility,product_solubility,...,product_single_point_job_id,reactant_optimization_job_id,product_optimization_job_id,UMAP-1,UMAP-2,data_type,reactantUFF,reactantMMFF,productUFF,productMMFF
767,769,25,NH,OH,c1cccc(c12)cc(nn2)O,c1cccc(c12)C=C(O)NN2,CXUGAWWYKSOLEL-UHFFFAOYSA-N,BMBWSQWNXPSELG-UHFFFAOYSA-N,-2.080,-2.410,...,2965.0,2369.0,2964.0,21.961576,-2.257458,2,20.955169,18.343820,17.140645,19.530730
772,774,25,NH,OH,Oc1cccc(c12)cc(nn2)O,Oc1cccc(c12)C=C(O)NN2,BSAMMELAAJYOKU-UHFFFAOYSA-N,QXPSFNRPQZDOFI-UHFFFAOYSA-N,-2.173,-2.142,...,3339.0,2335.0,3338.0,22.153984,-2.141872,2,22.612127,27.933289,20.633665,23.125357
776,778,25,NH,OH,c1c(O)ccc(c12)cc(nn2)O,c1c(O)ccc(c12)C=C(O)NN2,CDYBFEZYDGWUIQ-UHFFFAOYSA-N,DGADYJIUSCQUEV-UHFFFAOYSA-N,-1.883,-2.219,...,3027.0,2343.0,3026.0,22.034029,-2.341485,2,21.586312,3.518582,17.767113,7.922284
779,781,25,NH,OH,c1cc(O)cc(c12)cc(nn2)O,c1cc(O)cc(c12)C=C(O)NN2,IOXNOKWEMRWDTL-UHFFFAOYSA-N,KBWVQLXFEXJWTQ-UHFFFAOYSA-N,-1.883,-2.234,...,3179.0,2488.0,3178.0,21.784317,-2.370045,2,21.587192,5.173647,18.301297,6.773585
781,783,25,NH,OH,c1ccc(O)c(c12)cc(nn2)O,c1ccc(O)c(c12)C=C(O)NN2,KNEPRHJALMLDNQ-UHFFFAOYSA-N,COWUWBQUYWRHNQ-UHFFFAOYSA-N,-2.112,-2.162,...,3001.0,2558.0,3000.0,22.319105,-2.678297,2,24.719301,11.516633,21.434299,18.428400


In [5]:
train_encoded = get_ecfc(train_data["reactant_smiles"])
test1_encoded = get_ecfc(test1_data["reactant_smiles"])
test2_encoded = get_ecfc(test2_data["reactant_smiles"])

In [6]:
# def modeling(train_data, test1_data, test2_data, encoder, model):  - After deciding the list of the smiles

def modeling(train_encoded, test1_encoded, test2_encoded, model):
    
    from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
    import time
    start_time = time.time()
    
    # Training
    X = train_encoded
    y = train_data['reaction_energy']
    
    model.fit(X.values, y)
    
    # Predicting
    pred_train = model.predict(train_encoded.values)
    pred_test1 = model.predict(test1_encoded.values)
    pred_test2 = model.predict(test2_encoded.values)
    
    
    # Scores of Train Data 
    tr_mae = mean_absolute_error(y, pred_train)
    tr_rmse = mean_squared_error(y ,pred_train , squared=False)
    tr_r2 = r2_score(y, pred_train)
    print('##########################  Scores of Train Data  ##########################')
    print('Train set MAE of {}: {:.3f}'.format(model, tr_mae))
    print('Train set RMSE of {}: {:.3f}'.format(model, tr_rmse))
    print('Train set R2 Score of {}: {:.3f}'.format(model, tr_r2))
    
    print("----------------------------------------------------------------------------")
    
    # Test1 Data
    test1_mae = mean_absolute_error(test1_data['reaction_energy'], pred_test1)
    test1_rmse = mean_squared_error(test1_data['reaction_energy'], pred_test1, squared=False)
    test1_r2 = r2_score(test1_data['reaction_energy'], pred_test1)
    print('##########################  Scores of Test1 Data  ##########################')
    print('Test1 set MAE of {}: {:.3f}'.format(model, test1_mae))
    print('Test1 set RMSE of {}: {:.3f}'.format(model, test1_rmse))
    print('Test1 set R2 Score of {}: {:.3f}'.format(model, test1_r2))
    
    print("----------------------------------------------------------------------------")
    
    # Test2 Data
    test2_mae = mean_absolute_error(test2_data['reaction_energy'], pred_test2)
    test2_rmse = mean_squared_error(test2_data['reaction_energy'], pred_test2, squared=False)
    test2_r2 = r2_score(test2_data['reaction_energy'], pred_test2)
    print('##########################  Scores of Test2 Data  ##########################')
    print('Test2 set MAE of {}: {:.3f}'.format(model, test2_mae))
    print('Test2 set RMSE of {}: {:.3f}'.format(model, test2_rmse))
    print('Test2 set R2 Score of {}: {:.3f}'.format(model, test2_r2))
    
    print("----------------------------------------------------------------------------")

    elapsed_time = time.time() - start_time
    print('##########################  Details  ##########################')
    print(f'{elapsed_time:.2f}s elapsed during modeling')

# INITIAL RUNNING for RF

In [ ]:
Initial Default Values: {'n_estimators=': 100, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1}

In [7]:
from sklearn.ensemble import RandomForestRegressor

# Model
model = RandomForestRegressor(random_state=1)

# Training
modeling(train_encoded=train_encoded, test1_encoded=test1_encoded, test2_encoded=test2_encoded, model=model)

##########################  Scores of Train Data  ##########################
Train set MAE of RandomForestRegressor(random_state=1): 0.001
Train set RMSE of RandomForestRegressor(random_state=1): 0.003
Train set R2 Score of RandomForestRegressor(random_state=1): 0.993
----------------------------------------------------------------------------
##########################  Scores of Test1 Data  ##########################
Test1 set MAE of RandomForestRegressor(random_state=1): 0.004
Test1 set RMSE of RandomForestRegressor(random_state=1): 0.008
Test1 set R2 Score of RandomForestRegressor(random_state=1): 0.963
----------------------------------------------------------------------------
##########################  Scores of Test2 Data  ##########################
Test2 set MAE of RandomForestRegressor(random_state=1): 0.008
Test2 set RMSE of RandomForestRegressor(random_state=1): 0.011
Test2 set R2 Score of RandomForestRegressor(random_state=1): 0.853
---------------------------------------

In [ ]:
INITIAL SCORES: 0.001	0.003	0.993		0.004	0.008	0.963		0.008	0.011	0.853

In [8]:
model

RandomForestRegressor(random_state=1)

--------------

# Saving Test 1 Predictions

In [9]:
test1_data.head(2)

,reaction_id,data_package_id,bond_type,functional_group_stoichiometry,reactant_smiles,product_smiles,reactant_inchiKey,product_inchiKey,reactant_solubility,product_solubility,...,product_single_point_job_id,reactant_optimization_job_id,product_optimization_job_id,UMAP-1,UMAP-2,data_type,reactantUFF,reactantMMFF,productUFF,productMMFF
2,3,1,OH,COOH,O=C1CC(=O)C(C(=O)O)=C1C(=O)O,C1C(O)=C(C(=O)O)C(=C1O)C(=O)O,QLGSJNWSAMBDMI-UHFFFAOYSA-N,AEVQXUUICHCAGT-UHFFFAOYSA-N,-0.966,-0.867,...,26.0,19.0,25.0,13.362195,3.405288,1,33.280029,-113.415199,37.053325,-2.716193
3,4,1,OH,F,O=C1CC(=O)C=C1F,C1C(O)=C(F)C=C1O,XVCHEZRSXGJYDT-UHFFFAOYSA-N,LLYKNQJBRVSOKM-UHFFFAOYSA-N,-0.524,-0.499,...,34.0,21.0,33.0,14.856011,2.316219,1,23.112465,-12.882731,24.103198,34.474933


In [10]:
pred_test1 = model.predict(test1_encoded.values)
rf_result_test1 = test1_data[["reaction_id", "reactant_smiles", "reaction_energy"]]
rf_result_test1["pred_test1"] = pred_test1
rf_result_test1

C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,reaction_id,reactant_smiles,reaction_energy,pred_test1
2,3,O=C1CC(=O)C(C(=O)O)=C1C(=O)O,-0.06394,-0.041635
3,4,O=C1CC(=O)C=C1F,-0.01642,-0.010434
13,14,O=C1NC(=O)C=C1,-0.00731,0.003257
14,15,O=C1NC(=O)C(F)=C1F,-0.00118,0.009145
24,25,O=C1SC(=O)C(C(=O)O)=C1C(=O)O,-0.06775,-0.043905
...,...,...,...,...
15759,15897,O=S(=O)(O)c(c1)cc(S(=O)(=O)O)c(c12)c(O)n(c2O)N...,-0.00215,-0.002260
15762,15900,c1cccc(c12)c(O)n(c2O)N(C3=O)C(=O)c(c34)c(S(=O)...,0.02162,0.008751
15768,15906,O=S(=O)(O)c(c1)c(S(=O)(=O)O)c(S(=O)(=O)O)c(c12...,-0.01803,-0.011441
15788,15927,O=S(=O)(O)c1ccc(S(=O)(=O)O)c(c12)c(O)n(c2O)N(C...,-0.03881,-0.009339


In [11]:
pred_test1

array([-0.0416346 , -0.0104341 ,  0.0032572 , ..., -0.01144083,
       -0.0093393 , -0.0099856 ])

In [12]:
# saving the dataframe
rf_result_test1.to_csv(r'.\final_models\rf_result_test1.csv', index=False)

# Saving Test 2 Predictions

In [13]:
test2_data.head(2)

,reaction_id,data_package_id,bond_type,functional_group_stoichiometry,reactant_smiles,product_smiles,reactant_inchiKey,product_inchiKey,reactant_solubility,product_solubility,...,product_single_point_job_id,reactant_optimization_job_id,product_optimization_job_id,UMAP-1,UMAP-2,data_type,reactantUFF,reactantMMFF,productUFF,productMMFF
767,769,25,NH,OH,c1cccc(c12)cc(nn2)O,c1cccc(c12)C=C(O)NN2,CXUGAWWYKSOLEL-UHFFFAOYSA-N,BMBWSQWNXPSELG-UHFFFAOYSA-N,-2.080,-2.410,...,2965.0,2369.0,2964.0,21.961576,-2.257458,2,20.955169,18.343820,17.140645,19.530730
772,774,25,NH,OH,Oc1cccc(c12)cc(nn2)O,Oc1cccc(c12)C=C(O)NN2,BSAMMELAAJYOKU-UHFFFAOYSA-N,QXPSFNRPQZDOFI-UHFFFAOYSA-N,-2.173,-2.142,...,3339.0,2335.0,3338.0,22.153984,-2.141872,2,22.612127,27.933289,20.633665,23.125357


In [14]:
pred_test2 = model.predict(test2_encoded.values)
rf_result_test2 = test2_data[["reaction_id", "reactant_smiles", "reaction_energy"]]
rf_result_test2["pred_test2"] = pred_test2
rf_result_test2

C:\Users\PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,reaction_id,reactant_smiles,reaction_energy,pred_test2
767,769,c1cccc(c12)cc(nn2)O,-0.01916,-0.015394
772,774,Oc1cccc(c12)cc(nn2)O,-0.01587,-0.013407
776,778,c1c(O)ccc(c12)cc(nn2)O,-0.01793,-0.013855
779,781,c1cc(O)cc(c12)cc(nn2)O,-0.01559,-0.013422
781,783,c1ccc(O)c(c12)cc(nn2)O,-0.01734,-0.013825
...,...,...,...,...
14233,14338,O=C(O)c1cc(C(=O)O)c(C(=O)O)c(c12)S/C(C2=O)=C(C...,-0.06081,-0.056273
14234,14339,O=C(O)c1c(C(=O)O)cc(C(=O)O)c(c12)S/C(C2=O)=C(C...,-0.04998,-0.050095
14235,14340,O=C(O)c1c(C(=O)O)cc(C(=O)O)c(c12)S/C(C2=O)=C(C...,-0.05763,-0.050680
14236,14341,O=C(O)c1c(C(=O)O)c(C(=O)O)cc(c12)S/C(C2=O)=C(C...,-0.05533,-0.050916


In [15]:
pred_test2

array([-0.0153938 , -0.0134067 , -0.0138553 , ..., -0.0506797 ,
       -0.05091561, -0.0521688 ])

In [16]:
# saving the dataframe
rf_result_test2.to_csv(r'.\final_models\rf_result_test2.csv', index=False)


--------------------------------------------------------------------------------------------------------------

# Saving RF Model

In [17]:
model

RandomForestRegressor(random_state=1)

In [18]:
pred_test1 = model.predict(test1_encoded.values)
pred_test1

array([-0.0416346 , -0.0104341 ,  0.0032572 , ..., -0.01144083,
       -0.0093393 , -0.0099856 ])

In [25]:
#model.save_model(r'.\final_models\xgbm_final_model.txt')

In [19]:
import pickle
# save
pickle.dump(model, open(r'.\final_models\rf_final_model.txt', "wb"))

### Read Saved model 

In [23]:
# load
rf_final_model = pickle.load(open(r'.\final_models\rf_final_model.txt', "rb"))

In [24]:
rf_final_model

RandomForestRegressor(random_state=1)

In [25]:
s_pred_test1 = rf_final_model.predict(test1_encoded.values)
s_pred_test1

array([-0.0416346 , -0.0104341 ,  0.0032572 , ..., -0.01144083,
       -0.0093393 , -0.0099856 ])